In [5]:
import xgboost as xgb
import pandas as pd
import numpy as np
import scipy as sc

In [6]:
train_set = pd.read_csv('dataset/output/train_local_dataset_polimi_extended_no2014.csv',delimiter=',')
test_set = pd.read_csv('dataset/output/test_local_dataset_polimi_extended_no2014.csv',delimiter=',')

In [7]:
y_train = train_set.pop('vendite')
t_train = test_set.pop('vendite')

In [8]:
xgdmat = xgb.DMatrix(train_set,y_train, missing=np.nan)
dtest = xgb.DMatrix(test_set, t_train, missing=np.nan)

In [10]:
def mape(predictions, dtrain):
    ground_truth = dtrain.get_label()
    one_over_n = 1/len(predictions)
    at_mean = sum(ground_truth)/len(ground_truth)
    sommatoria = sum(np.abs(ground_truth - predictions)) / at_mean
    return 'mape', one_over_n * sommatoria

In [11]:
our_params = {'eta': 0.1, 'seed':0, 'subsample': 0.8, 'colsample_bytree': 0.8, 
             'objective': 'reg:linear', 'max_depth':10, 'min_child_weight':10} 

# Questo dictionary verra popolato con tutti i risultati della MAPE del train e 
# test set computati ad ogni iterazione
dic = {}

# Ad ogni iteraziore, voglio 'guardare' questi due dataset e valutarli con la funzione
# passata in feval
watchlist = [(xgdmat,'train'),(dtest,'test')]

final_gb = xgb.train(our_params, xgdmat, 562, evals = watchlist, feval = mape, evals_result = dic)

[0]	train-mape:0.821033	test-mape:0.817743
[1]	train-mape:0.763152	test-mape:0.764042
[2]	train-mape:0.697718	test-mape:0.692788
[3]	train-mape:0.641265	test-mape:0.630651
[4]	train-mape:0.594023	test-mape:0.576884
[5]	train-mape:0.565063	test-mape:0.54896
[6]	train-mape:0.52656	test-mape:0.507676
[7]	train-mape:0.49524	test-mape:0.474188
[8]	train-mape:0.467845	test-mape:0.446267
[9]	train-mape:0.452539	test-mape:0.433668
[10]	train-mape:0.431863	test-mape:0.416855
[11]	train-mape:0.412767	test-mape:0.402423
[12]	train-mape:0.397329	test-mape:0.385591
[13]	train-mape:0.38131	test-mape:0.372867
[14]	train-mape:0.370672	test-mape:0.362608
[15]	train-mape:0.358856	test-mape:0.352422
[16]	train-mape:0.348966	test-mape:0.344747
[17]	train-mape:0.341133	test-mape:0.340492
[18]	train-mape:0.333069	test-mape:0.333006
[19]	train-mape:0.326675	test-mape:0.331096
[20]	train-mape:0.322232	test-mape:0.326573
[21]	train-mape:0.314768	test-mape:0.318392
[22]	train-mape:0.311488	test-mape:0.31831
[23

In [12]:
min(dic['train']['mape'])

0.170672

In [13]:
min(dic['test']['mape'])

0.269912